In [6]:
import pandas as pd
import numpy as np
import math as math
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.feature_selection import VarianceThreshold
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import CountVectorizer

# Importing datasets

Dataset with final stats and dataset with players' efficiency indexes have to be merged in order to analysis results.

In [7]:
dic_eff_basic = pd.read_csv('D:\Luka Sve\Strucni radovi\DEA\Rezultati - tabele\Rezultati basic DEA/dic_efficiency_basic.csv')
final_stats = pd.read_csv('D:\Luka Sve\Strucni radovi\DEA/final_stats.csv')

In [8]:
final_stats.head()

,player_name,club,position,minutes_played,shots_on_target,dribbles,pass_completed,tackles_won,clearance_attempted,goals,assists,balls_recoverd
0,Lisandro Martínez,Ajax,Defender,720,2,3,633,2,19,0,1,53
1,Mazraoui,Ajax,Defender,661,3,13,338,10,9,0,2,40
2,Blind,Ajax,Defender,572,2,8,368,4,8,1,1,32
3,Haller,Ajax,Forward,668,15,9,134,2,5,11,1,10
4,Tadić,Ajax,Forward,630,3,13,231,0,1,2,2,12


In [9]:
dic_eff_basic = dic_eff_basic.rename(columns={"Unnamed: 0": "player_name"}).set_index('player_name')
final_stats = final_stats.set_index('player_name')

In [10]:
dic_eff_basic = pd.merge(dic_eff_basic, final_stats[['club','position']], on="player_name", how='outer')

In [11]:
dic_eff_basic.head(20)

,Index,Efficiency,Slack,club,position
player_name,,,,,
Lisandro Martínez,1.318593,0.758384,0.318593,Ajax,Defender
Mazraoui,1.400693,0.713932,0.400693,Ajax,Defender
Blind,1.541669,0.648648,0.541669,Ajax,Defender
Haller,1.000000,1.000000,0.000000,Ajax,Forward
Tadić,1.083333,0.923077,0.083333,Ajax,Forward
Berghuis,1.638902,0.610165,0.638902,Ajax,Forward
Antony,1.079051,0.926740,0.079051,Ajax,Forward
Neres,1.219219,0.820197,0.219219,Ajax,Forward
Gravenberch,2.030151,0.492574,1.030151,Ajax,Midfielder


# Efficient players

Players with efficiency index of 1.
There are 94 of them in basic DEA model.

In [12]:
eff_players = dic_eff_basic[dic_eff_basic['Efficiency'] == 1]
eff_players

,Index,Efficiency,Slack,club,position
player_name,,,,,
Haller,1.0,1.0,0.0,Ajax,Forward
Klaassen,1.0,1.0,0.0,Ajax,Midfielder
Palomino,1.0,1.0,0.0,Atalanta,Defender
Djimsiti,1.0,1.0,0.0,Atalanta,Defender
Pašalić,1.0,1.0,0.0,Atalanta,Midfielder
...,...,...,...,...,...
Arnold,1.0,1.0,0.0,Wolfsburg,Midfielder
Hefti,1.0,1.0,0.0,Young Boys,Defender
Azmoun,1.0,1.0,0.0,Zenit,Forward


# Inefficient players

Inefficient players have index of less than 1 and there are 347 of them.

In [13]:
ineff_players = dic_eff_basic[dic_eff_basic['Efficiency'] != 1]
ineff_players

,Index,Efficiency,Slack,club,position
player_name,,,,,
Lisandro Martínez,1.318593,0.758384,0.318593,Ajax,Defender
Mazraoui,1.400693,0.713932,0.400693,Ajax,Defender
Blind,1.541669,0.648648,0.541669,Ajax,Defender
Tadić,1.083333,0.923077,0.083333,Ajax,Forward
Berghuis,1.638902,0.610165,0.638902,Ajax,Forward
...,...,...,...,...,...
Malcom,2.064022,0.484491,1.064022,Zenit,Forward
Barrios,1.464926,0.682628,0.464926,Zenit,Midfielder
Claudinho,1.613312,0.619843,0.613312,Zenit,Midfielder


In [14]:
# The most inefficient player - Depay

ineff_players[ineff_players['Efficiency'] == ineff_players['Efficiency'].min()]

,Index,Efficiency,Slack,club,position
player_name,,,,,
Depay,10.423446,0.095938,9.423446,Barcelona,Forward


In [15]:
# Couple of the most inefficient players

ineff_players[ineff_players['Efficiency'] < 0.25]

,Index,Efficiency,Slack,club,position
player_name,,,,,
Depay,10.423446,0.095938,9.423446,Barcelona,Forward
Evanilson,4.435577,0.225450,3.435577,Porto,Forward
Munir,5.194911,0.192496,4.194911,Sevilla,Forward
Marlos,4.608533,0.216989,3.608533,Shakhtar Donetsk,Midfielder


# Efficiency by clubs

It is interesting analysis of clubs' efficiency. Thus, for every club it can be seen the number of efficient players.

In [16]:
pd.DataFrame(eff_players.groupby('club').count().iloc[:,1])

,Efficiency
club,
Ajax,2
Atalanta,5
Atlético,2
Barcelona,3
Bayern,2
Benfica,5
Beşiktaş,2
Chelsea,5
Club Brugge,1


In the next line, we observe percentage of players' efficiency by clubs.

In [17]:
perc_eff = pd.DataFrame(eff_players.groupby('club').count()['Efficiency'] / dic_eff_basic.groupby('club').count()['Efficiency'])
perc_eff

,Efficiency
club,
Ajax,0.166667
Atalanta,0.357143
Atlético,0.111111
Barcelona,0.250000
Bayern,0.133333
Benfica,0.357143
Beşiktaş,0.153846
Chelsea,0.294118
Club Brugge,0.100000


In [18]:
# These are the best ranked clubs for this statistics

perc_eff[perc_eff['Efficiency'] > 0.35]

,Efficiency
club,
Atalanta,0.357143
Benfica,0.357143
Dortmund,0.500000
Salzburg,0.357143


# Inefficiency by clubs

Inefficient players grouped by clubs.

In [19]:
pd.DataFrame(ineff_players.groupby('club').count().iloc[:, 1])

,Efficiency
club,
Ajax,10
Atalanta,9
Atlético,16
Barcelona,9
Bayern,13
Benfica,9
Beşiktaş,11
Chelsea,12
Club Brugge,9


In [20]:
# The worst ranked clubs by efficiency

perc_eff[perc_eff['Efficiency'] < 0.1]

,Efficiency
club,
Inter,0.083333
Malmö,0.076923
Paris,0.071429
Young Boys,0.083333


# Role models dataset

Every inefficient player has his role model player or players, which has/have good generated output based on his/their input resources. Thus, it will be shown for every player his role models and the most frequent role models will be highlighted.

In [21]:
dic_role_models = pd.read_csv('D:\Luka Sve\Strucni radovi\DEA\Rezultati - tabele\Rezultati basic DEA/dic_role_models_basic.csv')
dic_role_models = dic_role_models.rename(columns={"Unnamed: 0": "player_name"}).set_index('player_name')
dic_role_models.head(10)

,Role_model
player_name,
Lisandro Martínez,"Djimsiti, Pjanic, Dahoud, Koundé, Fernando"
Mazraoui,"Djimsiti, Dahoud, Gvardiol, Uribe"
Blind,"Djimsiti, Pjanic, Werner, Dahoud, Uribe, Fernando"
Haller,/
Tadić,"Munir, Werner, Pulišić, Dahoud, Kolovos, Nuno ..."
Berghuis,"Pjanic, Werner, Pulišić, Angeliño, Firmino, Fe..."
Antony,"Werner, Dahoud, Gvardiol, Bruno Fernandes, Uribe"
Neres,"Lewandowski, Malen, Brandt, Minamino, Firmino"
Gravenberch,"Djimsiti, Dahoud, Uribe, Solet, Fernando"


For the purpose of extracting role models it will be used CountVectorizer function.

In [23]:
corpus = dic_role_models['Role_model']

vect = CountVectorizer(ngram_range=(1,1))
vect.fit_transform(corpus)

<441x177 sparse matrix of type '<class 'numpy.int64'>'
	with 1868 stored elements in Compressed Sparse Row format>

In [24]:
# There is 177 players, who are role models to someone

len(set(vect.get_feature_names()))

177

In [25]:
role_models_max = pd.DataFrame(vect.fit_transform(corpus).todense(), columns=vect.get_feature_names())

In [26]:
# The check for number of role models

role_models_max.sum(axis=1)

0      5
1      4
2      6
3      0
4      9
      ..
436    0
437    5
438    6
439    6
440    4
Length: 441, dtype: int64

In [27]:
# There is 6 players with more than 75 time being role model

sum(role_models_max.sum(axis=0) > 75)

6

In [28]:
role_models_max = pd.DataFrame(role_models_max.sum(axis=0),columns={"number"})

In [39]:
# These are the players with the highest number of time of being role model to someone

role_models_max = role_models_max[role_models_max['number'] > 75]
role_models_max.index.name = "player_name"
role_models_max.index = role_models_max.index.str.capitalize()
role_models_max

,number
player_name,
Dahoud,76
Djimsiti,98
Fernando,152
Solet,98
Uribe,141
Werner,77


# Normalized players' weights/coefficients of virtual variables

Virtual variable coefficients must be examined in order to determine which input variables can be improved in terms of improving the efficiency index.

In [31]:
dic_norm_weights = pd.read_csv('D:\Luka Sve\Strucni radovi\DEA\Rezultati - tabele\Rezultati basic DEA/weights_norm_basic.csv')
dic_norm_weights = dic_norm_weights.rename(columns={"Unnamed: 0": "player_name"}).set_index('player_name')
dic_norm_weights = round(dic_norm_weights, 3)
dic_norm_weights

,minutes_played,shots_on_target,dribbles,pass_completed,tackles_won,clearance_attempted,goals,assists,balls_recoverd
player_name,,,,,,,,,
Lisandro Martínez,1.100,0.047,0.000,0.000,0.044,0.128,0.000,0.078,0.922
Mazraoui,1.081,0.000,0.000,0.241,0.000,0.078,0.000,0.214,0.786
Blind,1.239,0.072,0.000,0.000,0.072,0.160,0.073,0.106,0.820
Haller,0.111,0.733,0.016,0.140,0.000,0.000,1.000,0.000,0.000
Tadić,-0.000,1.000,0.000,0.000,0.000,0.083,0.667,0.333,-0.000
...,...,...,...,...,...,...,...,...,...
Kuzyaev,-0.000,1.000,0.000,-0.000,0.000,-0.000,1.000,0.000,-0.000
Barrios,0.956,0.000,0.000,0.228,0.146,0.135,0.000,0.000,1.000
Claudinho,0.796,0.000,0.000,0.000,0.325,0.492,0.120,0.000,0.880


In [32]:
dic_norm_weights = pd.merge(dic_norm_weights, dic_eff_basic[['Efficiency', 'club']], on="player_name", how="outer")
dic_norm_weights

,minutes_played,shots_on_target,dribbles,pass_completed,tackles_won,clearance_attempted,goals,assists,balls_recoverd,Efficiency,club
player_name,,,,,,,,,,,
Lisandro Martínez,1.100,0.047,0.000,0.000,0.044,0.128,0.000,0.078,0.922,0.758384,Ajax
Mazraoui,1.081,0.000,0.000,0.241,0.000,0.078,0.000,0.214,0.786,0.713932,Ajax
Blind,1.239,0.072,0.000,0.000,0.072,0.160,0.073,0.106,0.820,0.648648,Ajax
Haller,0.111,0.733,0.016,0.140,0.000,0.000,1.000,0.000,0.000,1.000000,Ajax
Tadić,-0.000,1.000,0.000,0.000,0.000,0.083,0.667,0.333,-0.000,0.923077,Ajax
...,...,...,...,...,...,...,...,...,...,...,...
Kuzyaev,-0.000,1.000,0.000,-0.000,0.000,-0.000,1.000,0.000,-0.000,1.000000,Zenit
Barrios,0.956,0.000,0.000,0.228,0.146,0.135,0.000,0.000,1.000,0.682628,Zenit
Claudinho,0.796,0.000,0.000,0.000,0.325,0.492,0.120,0.000,0.880,0.619843,Zenit


It will be analyzed a couple of playuers and theirs coefficients.

In [34]:
round(dic_norm_weights[dic_norm_weights.index == "Modrić"], 3)

,minutes_played,shots_on_target,dribbles,pass_completed,tackles_won,clearance_attempted,goals,assists,balls_recoverd,Efficiency,club
player_name,,,,,,,,,,,
Modrić,0.95,0.051,0.532,0.0,0.071,0.079,0.0,0.527,0.473,0.594,Real Madrid


In [35]:
round(dic_norm_weights[dic_norm_weights.index == "Lewandowski"], 3)

,minutes_played,shots_on_target,dribbles,pass_completed,tackles_won,clearance_attempted,goals,assists,balls_recoverd,Efficiency,club
player_name,,,,,,,,,,,
Lewandowski,0.125,0.871,0.004,0.0,0.0,0.0,1.0,0.0,0.0,1.0,Bayern


In [36]:
round(dic_norm_weights[dic_norm_weights.index == "Werner"], 3)

,minutes_played,shots_on_target,dribbles,pass_completed,tackles_won,clearance_attempted,goals,assists,balls_recoverd,Efficiency,club
player_name,,,,,,,,,,,
Werner,0.0,1.0,0.0,-0.0,0.0,0.0,0.8,0.2,0.0,1.0,Chelsea
